In [22]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version:  2.11.0


### Conjunto de Testes - MNIST

In [23]:
mnist = tf.keras.datasets.mnist

# Separação treino e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data();
x_train, x_test = x_train / 255.0, x_test /255.0;

#
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [24]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [37]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__();
        self.conv1 = Conv2D(32, 3, activation = 'relu');
        self.flatten = Flatten();
        self.d1 = Dense(128, activation = 'relu');
        self.d2 = Dense(10);
        
    def call(self, x):
        x = self.conv1(x);
        x = self.flatten(x);
        x = self.d1(x);
        return self.d2(x)

model = MyModel()

In [38]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True);
optimizer = tf.keras.optimizers.Adam();

In [39]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss');
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy');

test_loss = tf.keras.metrics.Mean(name = 'test_loss');
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy');

In [40]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True);
        loss = loss_object(labels, predictions);
    
    gradients = tape.gradient(loss, model.trainable_variables);
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss);
    train_accuracy(labels, predictions);

In [41]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training = False);
    t_loss = loss_object(labels, predictions);
    
    test_loss(t_loss);
    test_accuracy(labels, predictions);

In [44]:
EPOCHS = 2;

for epoch in range(EPOCHS):
    train_loss.reset_states();
    train_accuracy.reset_states();
    test_loss.reset_states();
    test_accuracy.reset_states();
    
    for images, labels in train_ds:
        train_step(images, labels);
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels);
    
    print(
    f'Epoch {epoch + 1},\n'
    f'Loss: {train_loss.result()},\n'
    f'Accuracy: {train_accuracy.result() * 100},\n'
    f'Test Loss: {test_loss.result()},\n'
    f'Test Accuracy: {test_accuracy.result() * 100},'
    
    )

Epoch 1,
Loss: 0.014036069624125957,
Accuracy: 99.5250015258789,
Test Loss: 0.06888333708047867,
Test Accuracy: 98.25,
Epoch 2,
Loss: 0.009237987920641899,
Accuracy: 99.6883316040039,
Test Loss: 0.0867333933711052,
Test Accuracy: 97.97999572753906,
